# Лабораторная работа №3

ФИО: Гучигов Джамбек Аюбович.

Группа: БИВТ-22-1.

## 1. Импорт данных.

In [1]:
import pandas as pd

data = pd.read_csv('/Users/admin/Desktop/AI/NY-House-Dataset.csv')
data.head()


,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


In [2]:
# Рассчитаем 1-й и 3-й квартиль и межквартильный размах (IQR)
Q1 = data['PRICE'].quantile(0.25)
Q3 = data['PRICE'].quantile(0.75)
IQR = Q3 - Q1

# Уберем строки, где цена выходит за пределы (Q1 - 1.5 * IQR) и (Q3 + 1.5 * IQR)
df_without_outliers = data[(data['PRICE'] >= (Q1 - 1.5 * IQR)) & (data['PRICE'] <= (Q3 + 1.5 * IQR))]


## 2. Анализ данных
### Определите целевую переменную (PRICE) и признаки.

## 3. Предобработка данных

### Проверка пропусков

In [3]:
print(data.isnull().sum())


BROKERTITLE                    0
TYPE                           0
PRICE                          0
BEDS                           0
BATH                           0
PROPERTYSQFT                   0
ADDRESS                        0
STATE                          0
MAIN_ADDRESS                   0
ADMINISTRATIVE_AREA_LEVEL_2    0
LOCALITY                       0
SUBLOCALITY                    0
STREET_NAME                    0
LONG_NAME                      0
FORMATTED_ADDRESS              0
LATITUDE                       0
LONGITUDE                      0
dtype: int64


### Разделение выборки

In [4]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['PRICE'])  # убираем целевой столбец
y = data['PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Нормализация численных признаков

In [5]:
from sklearn.preprocessing import StandardScaler

numerical_cols = ['BEDS', 'BATH', 'PROPERTYSQFT', 'LATITUDE', 'LONGITUDE']
scaler = StandardScaler()

X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


### Кодирование категориальных признаков

In [6]:
X_train = X_train.drop(columns=['BROKERTITLE', 'ADDRESS', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'STREET_NAME', 'LONG_NAME', 'FORMATTED_ADDRESS'])

In [7]:
X_test = X_test.drop(columns=['BROKERTITLE', 'ADDRESS', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'STREET_NAME', 'LONG_NAME', 'FORMATTED_ADDRESS'])

In [8]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['TYPE', 'STATE', 'LOCALITY', 'SUBLOCALITY']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Кодирование категориальных признаков
X_train_encoded = pd.DataFrame(
    encoder.fit_transform(X_train[categorical_cols]),
    index=X_train.index,  # Сохраняем индексы исходного X_train
    columns=encoder.get_feature_names_out(categorical_cols)  # Имена столбцов
)

X_test_encoded = pd.DataFrame(
    encoder.transform(X_test[categorical_cols]),
    index=X_test.index,
    columns=encoder.get_feature_names_out(categorical_cols)
)

# Объединение закодированных данных с числовыми
X_train = X_train.drop(columns=categorical_cols).join(X_train_encoded)
X_test = X_test.drop(columns=categorical_cols).join(X_test_encoded)




## 4. Обучение моделей

In [9]:
# Линейная модель
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)

In [10]:
# Деревья решений
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=7)  # Попробуйте разные max_depth
dt.fit(X_train, y_train)

y_train_pred_dt = dt.predict(X_train)
y_test_pred_dt = dt.predict(X_test)

In [11]:
# K-ближайшие соседи
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)  # можно разные k
knn.fit(X_train, y_train)

y_train_pred_knn = knn.predict(X_train)
y_test_pred_knn = knn.predict(X_test)

In [12]:
# Случайный лес
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, max_depth=10)  # Попробуйте разные параметры
rf.fit(X_train, y_train)

y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

## 5. Метрики

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Функция для расчета MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Для каждой модели и её предсказаний
# Линейная модель
mae_lr_train = mean_absolute_error(y_train, y_train_pred_lr)
mae_lr_test = mean_absolute_error(y_test, y_test_pred_lr)

rmse_lr_train = np.sqrt(mean_squared_error(y_train, y_train_pred_lr))
rmse_lr_test = np.sqrt(mean_squared_error(y_test, y_test_pred_lr))

mape_lr_train = mean_absolute_percentage_error(y_train, y_train_pred_lr)
mape_lr_test = mean_absolute_percentage_error(y_test, y_test_pred_lr)

# Деревья решений
mae_dt_train = mean_absolute_error(y_train, y_train_pred_dt)
mae_dt_test = mean_absolute_error(y_test, y_test_pred_dt)

rmse_dt_train = np.sqrt(mean_squared_error(y_train, y_train_pred_dt))
rmse_dt_test = np.sqrt(mean_squared_error(y_test, y_test_pred_dt))

mape_dt_train = mean_absolute_percentage_error(y_train, y_train_pred_dt)
mape_dt_test = mean_absolute_percentage_error(y_test, y_test_pred_dt)

# K-ближайшие соседи
mae_knn_train = mean_absolute_error(y_train, y_train_pred_knn)
mae_knn_test = mean_absolute_error(y_test, y_test_pred_knn)

rmse_knn_train = np.sqrt(mean_squared_error(y_train, y_train_pred_knn))
rmse_knn_test = np.sqrt(mean_squared_error(y_test, y_test_pred_knn))

mape_knn_train = mean_absolute_percentage_error(y_train, y_train_pred_knn)
mape_knn_test = mean_absolute_percentage_error(y_test, y_test_pred_knn)

# Случайный лес
mae_rf_train = mean_absolute_error(y_train, y_train_pred_rf)
mae_rf_test = mean_absolute_error(y_test, y_test_pred_rf)

rmse_rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
rmse_rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

mape_rf_train = mean_absolute_percentage_error(y_train, y_train_pred_rf)
mape_rf_test = mean_absolute_percentage_error(y_test, y_test_pred_rf)

# Выводим результаты
print("Линейная регрессия:")
print(f"MAE (train): {mae_lr_train}, MAE (test): {mae_lr_test}")
print(f"RMSE (train): {rmse_lr_train}, RMSE (test): {rmse_lr_test}")
print(f"MAPE (train): {mape_lr_train}, MAPE (test): {mape_lr_test}")
print("\n")

print("Деревья решений:")
print(f"MAE (train): {mae_dt_train}, MAE (test): {mae_dt_test}")
print(f"RMSE (train): {rmse_dt_train}, RMSE (test): {rmse_dt_test}")
print(f"MAPE (train): {mape_dt_train}, MAPE (test): {mape_dt_test}")
print("\n")

print("K-ближайшие соседи:")
print(f"MAE (train): {mae_knn_train}, MAE (test): {mae_knn_test}")
print(f"RMSE (train): {rmse_knn_train}, RMSE (test): {rmse_knn_test}")
print(f"MAPE (train): {mape_knn_train}, MAPE (test): {mape_knn_test}")
print("\n")

print("Случайный лес:")
print(f"MAE (train): {mae_rf_train}, MAE (test): {mae_rf_test}")
print(f"RMSE (train): {rmse_rf_train}, RMSE (test): {rmse_rf_test}")
print(f"MAPE (train): {mape_rf_train}, MAPE (test): {mape_rf_test}")


Линейная регрессия:
MAE (train): 1214522.0127604166, MAE (test): 2.1642378346726824e+16
RMSE (train): 3806444.7313435865, RMSE (test): 3.0055100757781645e+17
MAPE (train): 121.16571730053731, MAPE (test): 3873091965951.1514


Деревья решений:
MAE (train): 744678.7363207766, MAE (test): 964362.8455672613
RMSE (train): 1793106.7255733435, RMSE (test): 3055751.054455653
MAPE (train): 70.99606029763041, MAPE (test): 60.85389351782533


K-ближайшие соседи:
MAE (train): 1147709.1364583333, MAE (test): 1589263.044398196
RMSE (train): 25937576.99777163, RMSE (test): 23288652.08403755
MAPE (train): 43.24563098222766, MAPE (test): 71.44160080403911


Случайный лес:
MAE (train): 712551.8753619284, MAE (test): 817384.6490048587
RMSE (train): 12858881.138891218, RMSE (test): 3019112.401153906
MAPE (train): 49.80171552580133, MAPE (test): 48.079215280637044
